In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [2]:
csv_file = "../Resources/customer_data.csv"
customer_data_df = pd.read_csv(csv_file)
customer_data_df.head()

,id,first_name,last_name,email,gender,car
0,1,Benetta,Cancott,bcancott0@studiopress.com,Female,Scion
1,2,Lilyan,Cherry,lcherry1@deliciousdays.com,Female,Chrysler
2,3,Ezekiel,Benasik,ebenasik2@wikia.com,Male,Mercedes-Benz
3,4,Kennedy,Atlay,katlay3@so-net.ne.jp,Male,Buick
4,5,Sanford,Salmen,ssalmen4@reuters.com,Male,Lincoln


### Create new data with select columns

In [3]:
new_customer_data_df = customer_data_df[['id', 'first_name', 'last_name']].copy()
new_customer_data_df.head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Store JSON data into a DataFrame

In [4]:
json_file = "../Resources/customer_location.json"
customer_location_df = pd.read_json(json_file)
customer_location_df.head()

,address,id,latitude,longitude,us_state
0,043 Mockingbird Place,1,39.1682,-86.5186,Indiana
1,4 Prentice Point,2,41.0938,-85.0707,Indiana
2,46 Derek Junction,3,32.7673,-96.7776,Texas
3,11966 Old Shore Place,4,39.0350,-94.3567,Missouri
4,5 Evergreen Circle,5,40.7808,-73.9772,New York


### Clean DataFrame

In [15]:
new_customer_location_df = customer_location_df[["address", "us_state"]].copy()
new_customer_location_df.head()

,address,us_state
0,043 Mockingbird Place,Indiana
1,4 Prentice Point,Indiana
2,46 Derek Junction,Texas
3,11966 Old Shore Place,Missouri
4,5 Evergreen Circle,New York


### Connect to local database

In [6]:
user_name = input('Enter user name:')
password = input('Password:')
rds_connection_string = "{0}:{1}@localhost:5432/customer_db".format(user_name, password)
engine = create_engine(f'postgresql://{rds_connection_string}')

Enter user name:testuser
Password:TestPassword


### Check for tables

In [7]:
engine.table_names()

['customer_name', 'customer_location']

### Use pandas to load csv converted DataFrame into database

In [9]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

### Use pandas to load json converted DataFrame into database

In [16]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.IntegrityError) null value in column "id" violates not-null constraint
DETAIL:  Failing row contains (null, 043 Mockingbird Place, Indiana).

[SQL: INSERT INTO customer_location (address, us_state) VALUES (%(address)s, %(us_state)s)]
[parameters: ({'address': '043 Mockingbird Place', 'us_state': 'Indiana'}, {'address': '4 Prentice Point', 'us_state': 'Indiana'}, {'address': '46 Derek Junction', 'us_state': 'Texas'}, {'address': '11966 Old Shore Place', 'us_state': 'Missouri'}, {'address': '5 Evergreen Circle', 'us_state': 'New York'}, {'address': '5304 Vidon Drive', 'us_state': 'Texas'}, {'address': '65 Texas Avenue', 'us_state': 'Ohio'}, {'address': '041 Eagle Crest Court', 'us_state': 'Texas'}  ... displaying 10 of 1000 total bound parameter sets ...  {'address': '33 Bunting Point', 'us_state': 'Nevada'}, {'address': '8389 Sommers Street', 'us_state': 'Oregon'})]
(Background on this error at: http://sqlalche.me/e/gkpj)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [12]:
pd.read_sql_query('select * from customer_name', con=engine).head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Confirm data has been added by querying the customer_location table

In [13]:
pd.read_sql_query('select * from customer_location', con=engine).head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York


In [14]:
pd.read_sql_query('''SELECT customer_name.id, customer_name.first_name, customer_name.last_name, customer_location.address, customer_location.us_state
                    FROM customer_name
                    JOIN customer_location
                    ON customer_name.id = customer_location.id;''', con=engine).head()

,id,first_name,last_name,address,us_state
0,1,Benetta,Cancott,043 Mockingbird Place,Indiana
1,2,Lilyan,Cherry,4 Prentice Point,Indiana
2,3,Ezekiel,Benasik,46 Derek Junction,Texas
3,4,Kennedy,Atlay,11966 Old Shore Place,Missouri
4,5,Sanford,Salmen,5 Evergreen Circle,New York
